# Compare k-mer abundance and presence/absence vs "traditional" single-cell RNA-seq processing

- 500 k-mers, with abundance, were hashed from ~50k cells in Tabula Muris using `sourmash`


## Load Tabula Muris Senis data with fixed annotations

In [2]:
import dask.dataframe as dd
import numpy as np
import pandas as pd

import khtools

%load_ext autoreload
%autoreload 2

In [3]:
import scanpy.api as sc
sc. logging.print_versions()

scanpy==1.3.2 anndata==0.6.11 numpy==1.14.6 scipy==1.2.1 pandas==0.24.2 scikit-learn==0.20.3 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


In [3]:
# %%time

# h5ad = '/home/olga/ibm_lg/tabula-muris-senis/maca-facs-re-processed-annotated-fixed-tissues.h5ad'

# tms = sc.read_h5ad(h5ad)
# tms

## Read 50k x 50k similarity matrix (~50GB csv file)

In [4]:
csv = '/home/olga/pureScratch/olgabot-maca/facs/sourmash_compare_k31_ignore_abundance.csv'
! ls -lha $csv

-rw-rw-r-- 1 olga olga 14G Aug  7 11:51 /home/olga/pureScratch/olgabot-maca/facs/sourmash_compare_k31_ignore_abundance.csv


Had to increase `sample` to be the number of bytes(?) per row for `dd.read_csv` to be happy

In [5]:
%time similarity = dd.read_csv(csv, sample=2048000)

CPU times: user 6min 53s, sys: 3.68 s, total: 6min 56s
Wall time: 6min 55s


In [ ]:
# similarity.iloc[:, :5].head()

In [15]:
# %time similarity.iloc[:, :5].head()

CPU times: user 45.5 s, sys: 436 ms, total: 45.9 s
Wall time: 45.2 s


,A1-B000127-3_38_F-1-1_S1,A1-B000126-3_39_F-1-1_S289,A1-B000167-3_56_F-1-1_S228,A1-B000168-3_57_F-1-1_S177,A1-B000412-3_56_F-1-1_S110
0,1.000,0.048,0.052,0.000,0.034
1,0.048,1.000,0.094,0.002,0.034
2,0.052,0.094,1.000,0.000,0.036
3,0.000,0.002,0.000,1.000,0.000
4,0.034,0.034,0.036,0.000,1.000


## Run UMAP

Need to do 1-similarity to get disimilarity

In [ ]:
2+3

In [ ]:

import umap

In [ ]:
%time dissimilarity = 1-similarity

In [ ]:


embedding = umap.UMAP(n_neighbors=5, metric='precomputed', verbose=True).fit_transform(dissimilarity)

embedding

In [ ]:
dir()

In [9]:
embedding

NameError: name 'embedding' is not defined

In [ ]:
2+2

In [4]:
annotations = pd.read_csv('https://github.com/czbiohub/tabula-muris/raw/master/00_data_ingest/18_global_annotation_csv/annotations_facs.csv', 
                          index_col='cell')
annotations.index = annotations.index.str.replace('.', '-')
annotations.columns = annotations.columns.str.replace('.', '_')
annotations['sample_id'] = annotations.index
annotations = annotations.fillna("NA")
print(annotations.shape)
annotations.head()

/home/olga/miniconda3/envs/kmer-hashing/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(44949, 24)


,Neurog3>0_raw,Neurog3>0_scaled,cell_ontology_class,cell_ontology_id,cluster_ids,free_annotation,mouse_id,mouse_sex,plate_barcode,subsetA,...,subsetC_cluster_ids,subsetD,subsetD_cluster_ids,subsetE,subsetE_cluster_ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2,sample_id
cell,,,,,,,,,,,,,,,,,,,,,
A1-B000610-3_56_F-1-1,NA,NA,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_56_F,F,B000610,NA,...,NA,NA,NA,NA,NA,NA,Bladder,-7.182841,-10.895094,A1-B000610-3_56_F-1-1
A1-B002764-3_38_F-1-1,NA,NA,bladder urothelial cell,CL:1001428,3,Luminal bladder epithelial cell,3_38_F,F,B002764,NA,...,NA,NA,NA,NA,NA,NA,Bladder,4.199059,-11.050240,A1-B002764-3_38_F-1-1
A1-B002771-3_39_F-1-1,NA,NA,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_39_F,F,B002771,NA,...,NA,NA,NA,NA,NA,NA,Bladder,-11.995435,-7.325534,A1-B002771-3_39_F-1-1
A1-D041914-3_8_M-1-1,NA,NA,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_8_M,M,D041914,NA,...,NA,NA,NA,NA,NA,NA,Bladder,-6.820022,-14.174246,A1-D041914-3_8_M-1-1
A1-D042253-3_9_M-1-1,NA,NA,bladder cell,CL:1001319,2,Bladder mesenchymal cell,3_9_M,M,D042253,NA,...,NA,NA,NA,NA,NA,NA,Bladder,-24.163538,-7.499349,A1-D042253-3_9_M-1-1


In [9]:
cells = ['A1-B001717-3_38_F-1-1', 'A1-B003279-3_38_F-1-1', 'A1-D042103-3_11_M-1', 'A1-D042105-3_11_M-1-1', 'A1-MAA000427-3_11_M-1-',
         'A1-B000610-3_56_F-1-1']

annotations.loc[cells]

/home/olga/miniconda3/envs/kmer-hashing/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  after removing the cwd from sys.path.


,Neurog3>0_raw,Neurog3>0_scaled,cell_ontology_class,cell_ontology_id,cluster_ids,free_annotation,mouse_id,mouse_sex,plate_barcode,subsetA,...,subsetC_cluster_ids,subsetD,subsetD_cluster_ids,subsetE,subsetE_cluster_ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2,sample_id
cell,,,,,,,,,,,,,,,,,,,,,
A1-B001717-3_38_F-1-1,NA,NA,macrophage,CL:0000235,8.0,NA,3_38_F,F,B001717,NA,...,NA,NA,NA,NA,NA,NA,Kidney,-21.029031,-1.596872,A1-B001717-3_38_F-1-1
A1-B003279-3_38_F-1-1,NA,NA,microglial cell,CL:0000129,2.0,NA,3_38_F,F,B003279,NA,...,NA,NA,NA,NA,NA,Cerebellum,Brain_Myeloid,-31.706523,-17.767820,A1-B003279-3_38_F-1-1
A1-D042103-3_11_M-1,NA,NA,skeletal muscle satellite cell,CL:0000594,0.0,NA,3_11_M,M,D042103,NA,...,NA,NA,NA,NA,NA,ForelimbandHindlimb,Limb_Muscle,-22.391487,-2.537725,A1-D042103-3_11_M-1
A1-D042105-3_11_M-1-1,NA,NA,skeletal muscle satellite stem cell,CL:0008011,0.0,NA,3_11_M,M,D042105,NA,...,NA,NA,NA,NA,NA,NA,Diaphragm,-23.665230,-7.736316,A1-D042105-3_11_M-1-1
A1-MAA000427-3_11_M-1-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A1-B000610-3_56_F-1-1,NA,NA,bladder cell,CL:1001319,5.0,Bladder mesenchymal cell,3_56_F,F,B000610,NA,...,NA,NA,NA,NA,NA,NA,Bladder,-7.182841,-10.895094,A1-B000610-3_56_F-1-1


In [11]:
annotations.query('cell_ontology_class == "neuron"')

,Neurog3>0_raw,Neurog3>0_scaled,cell_ontology_class,cell_ontology_id,cluster_ids,free_annotation,mouse_id,mouse_sex,plate_barcode,subsetA,...,subsetC_cluster_ids,subsetD,subsetD_cluster_ids,subsetE,subsetE_cluster_ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2,sample_id
cell,,,,,,,,,,,,,,,,,,,,,
A1-MAA000940-3_8_M-1-1,NA,NA,neuron,CL:0000540,9,inhibitory neurons,3_8_M,M,MAA000940,False,...,NA,NA,NA,NA,NA,Striatum,Brain_Non-Myeloid,42.231779,3.110596,A1-MAA000940-3_8_M-1-1
A11-MAA000564-3_10_M-1-1,NA,NA,neuron,CL:0000540,9,inhibitory neurons,3_10_M,M,MAA000564,False,...,NA,NA,NA,NA,NA,Striatum,Brain_Non-Myeloid,44.270664,1.881176,A11-MAA000564-3_10_M-1-1
A11-MAA000942-3_8_M-1-1,NA,NA,neuron,CL:0000540,9,inhibitory neurons,3_8_M,M,MAA000942,False,...,NA,NA,NA,NA,NA,Striatum,Brain_Non-Myeloid,42.304715,2.366185,A11-MAA000942-3_8_M-1-1
A12-MAA000615-3_10_M-1-1,NA,NA,neuron,CL:0000540,9,inhibitory neurons,3_10_M,M,MAA000615,False,...,NA,NA,NA,NA,NA,Striatum,Brain_Non-Myeloid,42.897307,2.221820,A12-MAA000615-3_10_M-1-1
A13-MAA000424-3_11_M-1-1,NA,NA,neuron,CL:0000540,6,excitatory neurons and some neuronal stem cells,3_11_M,M,MAA000424,False,...,NA,NA,NA,NA,NA,Cerebellum,Brain_Non-Myeloid,30.735696,-7.333612,A13-MAA000424-3_11_M-1-1
A13-MAA000615-3_10_M-1-1,NA,NA,neuron,CL:0000540,6,excitatory neurons and some neuronal stem cells,3_10_M,M,MAA000615,False,...,NA,NA,NA,NA,NA,Striatum,Brain_Non-Myeloid,16.789822,-6.862530,A13-MAA000615-3_10_M-1-1
A14-MAA000561-3_10_M-1-1,NA,NA,neuron,CL:0000540,6,excitatory neurons and some neuronal stem cells,3_10_M,M,MAA000561,False,...,NA,NA,NA,NA,NA,Cortex,Brain_Non-Myeloid,16.778600,-6.907609,A14-MAA000561-3_10_M-1-1
A14-MAA000942-3_8_M-1-1,NA,NA,neuron,CL:0000540,9,inhibitory neurons,3_8_M,M,MAA000942,False,...,NA,NA,NA,NA,NA,Striatum,Brain_Non-Myeloid,44.963949,2.113199,A14-MAA000942-3_8_M-1-1
A15-MAA000942-3_8_M-1-1,NA,NA,neuron,CL:0000540,9,inhibitory neurons,3_8_M,M,MAA000942,False,...,NA,NA,NA,NA,NA,Striatum,Brain_Non-Myeloid,45.427715,2.090607,A15-MAA000942-3_8_M-1-1


## Dimensionality reduction

### PCA on similarties

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50, copy=False)

similarities_pca = pca.fit_transform(similarity)
print(similarities_pca.shape)

In [18]:
from dask_ml.decomposition import PCA

pca = PCA(n_components=50, copy=False)

ModuleNotFoundError: No module named 'dask_ml'

In [ ]:
from sklearn.manifold import U

### Old way of doing nearest neighbor graphs

In [ ]:
# color_cols = ['tissue', 'subtissue', 'cell_ontology_class', 'mouse_id', 'mouse_sex', 'plate_barcode']
# palette_names = ['tab20', 'tab10', 'cubehelix', 'husl', 'Paired', 'Set2']
# palettes = dict(zip(color_cols, palette_names))
# palettes

# graphs = {}
# positions = {}

# key = 'k31_with_abundance'

# n_neighbors = 5
# figure_folder = 's3://kmer-hashing/tabula-muris/n_hashes=500/'
# figure_prefix = 'molecule=DNA_ksize=31_ignore-abundance=True'
# title = 'Tabula Muris'

# g, pos = khtools.knn.build_graph_and_plot(similarity, annotations, n_neighbors,
#                                           color_cols, palettes, figure_folder, figure_prefix, title)

# graphs[key] = g
# positions[key] = pos

In [ ]:
khtools.__version__

## Run Leiden Clustering


Below is excerpted from [scanpy's leiden clustering algorithm](https://github.com/theislab/scanpy/blob/master/scanpy/tools/_leiden.py)

In [6]:


try:
    import leidenalg
except ImportError:
    raise ImportError('Please install the leiden algorithm: `pip3 install leidenalg`.')

# convert it to igraph
g = scanpy.utils.get_igraph_from_adjacency(adjacency, directed=directed)
# flip to the default partition type if not overriden by the user
if partition_type is None:
    partition_type = leidenalg.RBConfigurationVertexPartition
# prepare find_partition arguments as a dictionary, appending to whatever the user provided
# it needs to be this way as this allows for the accounting of a None resolution
# (in the case of a partition variant that doesn't take it on input)

partition_kwargs = None

if partition_kwargs is None:
    partition_kwargs = {}
if use_weights:
    partition_kwargs['weights'] = np.array(g.es['weight']).astype(np.float64)
partition_kwargs['n_iterations'] = n_iterations
partition_kwargs['seed'] = random_state
if resolution is not None:
    partition_kwargs['resolution_parameter'] = resolution
# clustering proper
part = leidenalg.find_partition(g, partition_type, **partition_kwargs)
# store output into adata.obs
groups = np.array(part.membership)


ImportError: Please install the leiden algorithm: `pip3 install leidenalg`.

## Make confusion matrix with Tabula Muris Senis leiden clustering

In [ ]:
tms.obs['leiden'].head()